In [7]:
import requests
response = requests.get("https://api.llama.fi/protocols")
protocols = response.json()
for p in protocols:
    if p['name'].lower() in ['compound', 'curve']:
        print(f"Name: {p['name']}, Slug: {p['slug']}")

In [9]:
# Import libraries
import requests
import pandas as pd
from datetime import datetime

# Define protocols
protocols = {
    'uniswap': 'Uniswap',
    'aave': 'Aave',
    'maple': 'Maple',
    'compound': 'Compound',
    'lido': 'Lido',
    'eigenlayer': 'EigenLayer',
    'curve-finance': 'Curve',  # Updated slug
    'ethena': 'Ethena'
}

# Initialize an empty list to store data
tvl_data = []

# Loop through each protocol
for slug, name in protocols.items():
    print(f"Fetching TVL for {name}...")
    url = f"https://api.llama.fi/protocol/{slug}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        data = response.json()
        # Extract TVL history
        for entry in data['tvl']:
            tvl_data.append({
                'Protocol': name,
                'Date': datetime.fromtimestamp(entry['date']).strftime('%Y-%m-%d'),
                'TVL_USD': entry['totalLiquidityUSD']
            })
    except requests.exceptions.HTTPError as e:
        print(f"Failed to fetch data for {name}: {e}")
    except KeyError:
        print(f"No TVL data available for {name}")

# Convert to DataFrame
df = pd.DataFrame(tvl_data)

# Filter for Q1 2024 and Q1 2025
df['Date'] = pd.to_datetime(df['Date'])
df = df[(df['Date'].between('2024-01-01', '2024-03-31')) | (df['Date'].between('2025-01-01', '2025-03-31'))]

# Save to CSV
df.to_csv('defi_tvl_data.csv', index=False)
print("Data saved to defi_tvl_data.csv")

# Display first few rows
print(df.head())

Fetching TVL for Uniswap...
Fetching TVL for Aave...
Fetching TVL for Maple...
Fetching TVL for Compound...
Failed to fetch data for Compound: 400 Client Error: Bad Request for url: https://api.llama.fi/protocol/compound
Fetching TVL for Lido...
Fetching TVL for EigenLayer...
Fetching TVL for Curve...
Fetching TVL for Ethena...
Data saved to defi_tvl_data.csv
     Protocol       Date       TVL_USD
1885  Uniswap 2024-01-01  3.712661e+09
1886  Uniswap 2024-01-02  3.832802e+09
1887  Uniswap 2024-01-03  3.825482e+09
1888  Uniswap 2024-01-04  3.658153e+09
1889  Uniswap 2024-01-05  3.727836e+09


In [10]:
# Import libraries
import requests
import pandas as pd
from datetime import datetime

# Define protocols with correct slugs
protocols = {
    'uniswap': 'Uniswap',
    'aave': 'Aave',
    'maple': 'Maple',
    'compound-finance': 'Compound',  # Updated slug
    'lido': 'Lido',
    'eigenlayer': 'EigenLayer',
    'curve-finance': 'Curve',  # Updated slug
    'ethena': 'Ethena'
}

# Initialize an empty list to store data
tvl_data = []

# Loop through each protocol
for slug, name in protocols.items():
    print(f"Fetching TVL for {name}...")
    url = f"https://api.llama.fi/protocol/{slug}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        data = response.json()
        # Check if TVL data exists
        if 'tvl' in data and data['tvl']:
            for entry in data['tvl']:
                tvl_data.append({
                    'Protocol': name,
                    'Date': datetime.fromtimestamp(entry['date']).strftime('%Y-%m-%d'),
                    'TVL_USD': entry['totalLiquidityUSD']
                })
        else:
            print(f"No TVL data available for {name}")
    except requests.exceptions.HTTPError as e:
        print(f"Failed to fetch data for {name}: HTTP Error - {e}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data for {name}: Request Error - {e}")
    except KeyError as e:
        print(f"Failed to fetch data for {name}: KeyError - {e}")

# Convert to DataFrame
df = pd.DataFrame(tvl_data)

# Filter for Q1 2024 (2024-01-01 to 2024-03-31) and Q1 2025 (2025-01-01 to 2025-03-31)
df['Date'] = pd.to_datetime(df['Date'])
df = df[(df['Date'].between('2024-01-01', '2024-03-31')) | (df['Date'].between('2025-01-01', '2025-03-31'))]

# Remove duplicates, if any
df = df.drop_duplicates()

# Sort by Protocol and Date
df = df.sort_values(by=['Protocol', 'Date'])

# Save to CSV
df.to_csv('defi_tvl_data.csv', index=False)
print("Data saved to defi_tvl_data.csv")

# Display first few rows and data summary
print("\nData Preview:")
print(df.head())
print("\nProtocols Included:")
print(df['Protocol'].unique())
print("\nDate Range:")
print(f"Min Date: {df['Date'].min()}, Max Date: {df['Date'].max()}")

Fetching TVL for Uniswap...
Fetching TVL for Aave...
Fetching TVL for Maple...
Fetching TVL for Compound...
Fetching TVL for Lido...
Fetching TVL for EigenLayer...
Fetching TVL for Curve...
Fetching TVL for Ethena...
Data saved to defi_tvl_data.csv

Data Preview:
     Protocol       Date       TVL_USD
3711     Aave 2024-01-01  6.628393e+09
3712     Aave 2024-01-02  6.849093e+09
3713     Aave 2024-01-03  6.853216e+09
3714     Aave 2024-01-04  6.444211e+09
3715     Aave 2024-01-05  6.646273e+09

Protocols Included:
['Aave' 'Compound' 'Curve' 'EigenLayer' 'Ethena' 'Lido' 'Maple' 'Uniswap']

Date Range:
Min Date: 2024-01-01 00:00:00, Max Date: 2025-03-31 00:00:00
